In [1]:
import torch
import pandas as pd
import numpy as np
import torch.nn as nn
from sklearn.model_selection import train_test_split
from sklearn import metrics
import torch.optim as optim

In [2]:
sub_features = ['42 tGravityAcc-mean()-Y',
 '43 tGravityAcc-mean()-Z',
 '51 tGravityAcc-max()-Y',
 '52 tGravityAcc-max()-Z',
 '54 tGravityAcc-min()-Y',
 '55 tGravityAcc-min()-Z',
 '56 tGravityAcc-sma()',
 '58 tGravityAcc-energy()-Y',
 '59 tGravityAcc-energy()-Z',
 '475 fBodyGyro-bandsEnergy()-1,8',
 '483 fBodyGyro-bandsEnergy()-1,16',
 '559 angle(X,gravityMean)',
 '560 angle(Y,gravityMean)',
 '561 angle(Z,gravityMean)']

act_features = ['4 tBodyAcc-std()-X',
 '10 tBodyAcc-max()-X',
 '17 tBodyAcc-energy()-X',
 '90 tBodyAccJerk-max()-X',
 '202 tBodyAccMag-std()',
 '203 tBodyAccMag-mad()',
 '215 tGravityAccMag-std()',
 '216 tGravityAccMag-mad()',
 '266 fBodyAcc-mean()-X',
 '269 fBodyAcc-std()-X',
 '272 fBodyAcc-mad()-X',
 '282 fBodyAcc-energy()-X',
 '303 fBodyAcc-bandsEnergy()-1,8',
 '311 fBodyAcc-bandsEnergy()-1,16',
 '315 fBodyAcc-bandsEnergy()-1,24',
 '382 fBodyAccJerk-bandsEnergy()-1,8',
 '504 fBodyAccMag-std()',
 '505 fBodyAccMag-mad()',
 '509 fBodyAccMag-energy()']

input_shape = len(sub_features) + len(act_features)

In [3]:
def classifier_block(input_dim, output_dim):
    return nn.Sequential(
        nn.Linear(input_dim, output_dim),
        nn.Dropout(0.1),
        nn.LeakyReLU(0.05)
    )

class Activity_Classifier(nn.Module):
    def __init__(self, feature_dim = input_shape):
        super(Activity_Classifier, self).__init__()
        self.network = nn.Sequential(
            classifier_block(feature_dim, 25),
            classifier_block(25, 20),
            classifier_block(20, 15),
            classifier_block(15, 10),
            nn.Linear(10, 3)
        )
    def forward(self, x):
        return self.network(x)
    
class Subject_Classifier(nn.Module):
    def __init__(self, feature_dim = input_shape):
        super(Subject_Classifier, self).__init__()
        self.network = nn.Sequential(
            classifier_block(feature_dim, 25),
            classifier_block(25, 20),
            classifier_block(20, 15),
            classifier_block(15, 10),
            nn.Linear(10, 6)
        )
    def forward(self, x):
        return self.network(x)

In [4]:
#defines each generator layer
#input and output dimensions needed
def generator_block(input_dim, output_dim):
    return nn.Sequential(
        nn.Linear(input_dim, output_dim),
        nn.Dropout(0.1),
        nn.BatchNorm1d(output_dim),
        nn.ReLU(inplace = True)
    )

#returns n_samples of z_dim (number of dimensions of latent space) noise
def get_noise(n_samples, z_dim):
    return torch.randn(n_samples, z_dim)

#defines generator class
class Generator(nn.Module):
    def __init__(self, z_dim = 10, feature_dim = input_shape, hidden_dim = 128):
        super(Generator, self).__init__()
        self.gen = nn.Sequential(
            generator_block(z_dim, 80),
            generator_block(80, 60),
            generator_block(60, 50),
            nn.Linear(50, feature_dim),
            nn.Tanh()
        )
    def forward(self, noise):
        return self.gen(noise)

def get_act_matrix(batch_size, a_dim):
    indexes = np.random.randint(a_dim, size = batch_size)
    
    one_hot = np.zeros((len(indexes), indexes.max()+1))
    one_hot[np.arange(len(indexes)),indexes] = 1
    return torch.Tensor(indexes).long(), torch.Tensor(one_hot)
    
def get_usr_matrix(batch_size, u_dim):
    indexes = np.random.randint(u_dim, size = batch_size)
    
    one_hot = np.zeros((indexes.size, indexes.max()+1))
    one_hot[np.arange(indexes.size),indexes] = 1
    return torch.Tensor(indexes).long(), torch.Tensor(one_hot)

def load_model(model, model_name):
    model.load_state_dict(torch.load(f'../../../saved_models/{model_name}'))

In [5]:
#label is a list of integers specifying which labels to filter by
#users is a list of integers specifying which users to filter by
#y_label is a string, either "Activity" or "Subject" depending on what y output needs to be returned
def start_data(label, users, y_label, sub_features, act_features):
    #get the dataframe column names
    name_dataframe = pd.read_csv('../../../data/features.txt', delimiter = '\n', header = None)
    names = name_dataframe.values.tolist()
    names = [k for row in names for k in row] #List of column names

    data = pd.read_csv('../../../data/X_train.txt', delim_whitespace = True, header = None) #Read in dataframe
    data.columns = names #Setting column names
    
    X_train_1 = data[sub_features]
    X_train_2 = data[act_features]
    X_train = pd.concat([X_train_1, X_train_2], axis = 1)
    
    y_train_activity = pd.read_csv('../../../data/y_train.txt', header = None)
    y_train_activity.columns = ['Activity']
    
    y_train_subject = pd.read_csv('../../../data/subject_train.txt', header = None)
    y_train_subject.columns = ['Subject']
    
    GAN_data = pd.concat([X_train, y_train_activity, y_train_subject], axis = 1)
    GAN_data = GAN_data[GAN_data['Activity'].isin(label)]
    GAN_data = GAN_data[GAN_data['Subject'].isin(users)]
    
    X_train = GAN_data.iloc[:,:-2].values
    y_train = GAN_data[[y_label]].values
    
    return X_train, y_train.flatten()

# Activity Classifier

In [6]:
activities = [1, 3, 4]
users = [1, 3, 5, 7, 8, 11]

X, y = start_data(activities, users, "Activity", sub_features, act_features)

In [7]:
for k in range(len(y)):
    if y[k] == 1:
        y[k] = 0
    elif y[k] == 3:
        y[k] = 1
    else:
        y[k] = 2

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, shuffle = True)

model = Activity_Classifier()
lr = 0.001
n_epochs = 5000
batch_size = 250

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr = lr)

train_features = torch.tensor(X_train)
train_labels = torch.tensor(y_train)
test_features = torch.tensor(X_test)
test_labels = torch.tensor(y_test)

train_data = torch.utils.data.TensorDataset(train_features, train_labels)
test_data = torch.utils.data.TensorDataset(test_features, test_labels)

train_loader = torch.utils.data.DataLoader(train_data, batch_size = batch_size, shuffle = True)
test_loader = torch.utils.data.DataLoader(test_data, batch_size = len(test_labels), shuffle = True)

In [9]:
for epoch in range(n_epochs):
    total_loss = 0
    for batch in train_loader:
        features, labels = batch
        
        optimizer.zero_grad()
        preds = model(features.float())
        
        loss = criterion(preds, labels.long()) 
        loss.backward()
        
        optimizer.step()
        total_loss += loss.item()
        
    print(f'Epoch {epoch + 1}, Loss: {total_loss}, Final Batch Loss: {loss.item()}')

Epoch 1, Loss: 4.332762122154236, Final Batch Loss: 1.037563681602478
Epoch 2, Loss: 4.338975429534912, Final Batch Loss: 1.0544679164886475
Epoch 3, Loss: 4.314476132392883, Final Batch Loss: 1.0376667976379395
Epoch 4, Loss: 4.2819541692733765, Final Batch Loss: 1.0079344511032104
Epoch 5, Loss: 4.5537601709365845, Final Batch Loss: 1.292805552482605
Epoch 6, Loss: 4.26775050163269, Final Batch Loss: 1.0072368383407593
Epoch 7, Loss: 4.269147276878357, Final Batch Loss: 1.0190597772598267
Epoch 8, Loss: 4.480282664299011, Final Batch Loss: 1.2444655895233154
Epoch 9, Loss: 4.221273958683014, Final Batch Loss: 0.9904813170433044
Epoch 10, Loss: 4.4857810735702515, Final Batch Loss: 1.2732107639312744
Epoch 11, Loss: 4.167403876781464, Final Batch Loss: 0.9695670008659363
Epoch 12, Loss: 4.197650194168091, Final Batch Loss: 1.0093309879302979
Epoch 13, Loss: 4.229457378387451, Final Batch Loss: 1.0511400699615479
Epoch 14, Loss: 4.34416663646698, Final Batch Loss: 1.1905593872070312
Ep

Epoch 121, Loss: 1.4132584035396576, Final Batch Loss: 0.9430651068687439
Epoch 122, Loss: 0.5500411875545979, Final Batch Loss: 0.01973341777920723
Epoch 123, Loss: 0.6075681820511818, Final Batch Loss: 0.06171760708093643
Epoch 124, Loss: 0.478874983265996, Final Batch Loss: 0.009172679856419563
Epoch 125, Loss: 0.49207987543195486, Final Batch Loss: 0.006574547849595547
Epoch 126, Loss: 0.8608949780464172, Final Batch Loss: 0.3669470250606537
Epoch 127, Loss: 0.4847510326653719, Final Batch Loss: 0.023003624752163887
Epoch 128, Loss: 0.48078217916190624, Final Batch Loss: 0.016887200996279716
Epoch 129, Loss: 0.484311081469059, Final Batch Loss: 0.029603563249111176
Epoch 130, Loss: 0.4143781578168273, Final Batch Loss: 0.010253137908875942
Epoch 131, Loss: 0.5212156176567078, Final Batch Loss: 0.0078497976064682
Epoch 132, Loss: 0.4936432056128979, Final Batch Loss: 0.031728919595479965
Epoch 133, Loss: 0.37698125350289047, Final Batch Loss: 0.0029627259355038404
Epoch 134, Loss: 0

Epoch 242, Loss: 0.3219840005040169, Final Batch Loss: 0.02616121619939804
Epoch 243, Loss: 0.3664289303123951, Final Batch Loss: 0.05270138010382652
Epoch 244, Loss: 0.2888956156093627, Final Batch Loss: 0.0007508557755500078
Epoch 245, Loss: 0.30688963062129915, Final Batch Loss: 0.0016374287661165
Epoch 246, Loss: 0.3440200686454773, Final Batch Loss: 0.06379096210002899
Epoch 247, Loss: 0.29563168389722705, Final Batch Loss: 0.00453467620536685
Epoch 248, Loss: 0.30642257793806493, Final Batch Loss: 0.003742834320291877
Epoch 249, Loss: 0.3025062996894121, Final Batch Loss: 0.017730897292494774
Epoch 250, Loss: 0.328701376914978, Final Batch Loss: 0.008133858442306519
Epoch 251, Loss: 0.4488726481795311, Final Batch Loss: 0.1483456790447235
Epoch 252, Loss: 0.31199186434969306, Final Batch Loss: 0.002075543161481619
Epoch 253, Loss: 0.2609716388396919, Final Batch Loss: 0.0030122878961265087
Epoch 254, Loss: 0.24791712185833603, Final Batch Loss: 0.0014855550834909081
Epoch 255, Lo

Epoch 360, Loss: 0.37831997126340866, Final Batch Loss: 0.18254320323467255
Epoch 361, Loss: 0.2559818085283041, Final Batch Loss: 0.027572372928261757
Epoch 362, Loss: 0.25205791737698746, Final Batch Loss: 8.821448318485636e-06
Epoch 363, Loss: 0.270680001238361, Final Batch Loss: 0.00279357866384089
Epoch 364, Loss: 0.19401853493764065, Final Batch Loss: 0.00035696811391972005
Epoch 365, Loss: 0.237162932656247, Final Batch Loss: 1.3947389561508317e-05
Epoch 366, Loss: 0.22415489983541192, Final Batch Loss: 9.643566590966657e-05
Epoch 367, Loss: 0.21526717639062554, Final Batch Loss: 0.0005274811992421746
Epoch 368, Loss: 0.22458127047866583, Final Batch Loss: 0.010489686392247677
Epoch 369, Loss: 0.26798414811491966, Final Batch Loss: 0.03520386293530464
Epoch 370, Loss: 0.9511478170752525, Final Batch Loss: 0.7250995635986328
Epoch 371, Loss: 0.2147818819139502, Final Batch Loss: 6.615896563744172e-05
Epoch 372, Loss: 0.3960597049444914, Final Batch Loss: 0.018527893349528313
Epoc

Epoch 468, Loss: 0.2523161011922639, Final Batch Loss: 0.00032658010604791343
Epoch 469, Loss: 0.2188583230599761, Final Batch Loss: 0.007259420119225979
Epoch 470, Loss: 0.18027514548157342, Final Batch Loss: 0.0002134810492862016
Epoch 471, Loss: 0.26044802367687225, Final Batch Loss: 0.023387879133224487
Epoch 472, Loss: 0.23685843497514725, Final Batch Loss: 0.03975147381424904
Epoch 473, Loss: 0.2333893571048975, Final Batch Loss: 0.022410938516259193
Epoch 474, Loss: 0.21932229015146731, Final Batch Loss: 1.549708758830093e-05
Epoch 475, Loss: 0.22372510211425833, Final Batch Loss: 0.00034731553751043975
Epoch 476, Loss: 0.3844621889293194, Final Batch Loss: 0.15058661997318268
Epoch 477, Loss: 3.790855549275875, Final Batch Loss: 3.601966381072998
Epoch 478, Loss: 0.2257873160415329, Final Batch Loss: 0.0007366804056800902
Epoch 479, Loss: 0.4747392609715462, Final Batch Loss: 0.04000142216682434
Epoch 480, Loss: 0.47004711779300123, Final Batch Loss: 0.0006435230607166886
Epoch

...

Epoch 4430, Loss: 0.0010310084326192737, Final Batch Loss: 0.0
Epoch 4431, Loss: 0.002224717682111077, Final Batch Loss: 0.0
Epoch 4432, Loss: 0.006646316469407054, Final Batch Loss: 1.1920928244535389e-07
Epoch 4433, Loss: 0.004758162423968315, Final Batch Loss: 0.0
Epoch 4434, Loss: 0.010447588651459228, Final Batch Loss: 2.50339189733495e-06
Epoch 4435, Loss: 0.01070954039460048, Final Batch Loss: 0.0
Epoch 4436, Loss: 0.0012317724176611478, Final Batch Loss: 1.1920928244535389e-07
Epoch 4437, Loss: 0.015396953443996608, Final Batch Loss: 0.0
Epoch 4438, Loss: 0.003018230908764963, Final Batch Loss: 2.861018856492592e-06
Epoch 4439, Loss: 0.01866868195065763, Final Batch Loss: 0.0
Epoch 4440, Loss: 0.006338434759527445, Final Batch Loss: 0.0
Epoch 4441, Loss: 0.02152958540500549, Final Batch Loss: 1.9192511899746023e-05
Epoch 4442, Loss: 0.0020098059903546073, Final Batch Loss: 2.3841855067985307e-07
Epoch 4443, Loss: 0.018555945251137018, Final Batch Loss: 0.0
Epoch 4444, Loss: 0.0

Epoch 4558, Loss: 0.011647865292616189, Final Batch Loss: 0.0
Epoch 4559, Loss: 0.005111587990541011, Final Batch Loss: 0.0
Epoch 4560, Loss: 0.006079699611291289, Final Batch Loss: 0.0
Epoch 4561, Loss: 0.001984546950545507, Final Batch Loss: 1.1920928244535389e-07
Epoch 4562, Loss: 0.026944729557726532, Final Batch Loss: 0.0
Epoch 4563, Loss: 0.00688391097355634, Final Batch Loss: 0.0
Epoch 4564, Loss: 0.0031684309942647815, Final Batch Loss: 0.0
Epoch 4565, Loss: 0.028359495423501357, Final Batch Loss: 3.659658250398934e-05
Epoch 4566, Loss: 0.03401258162193699, Final Batch Loss: 0.0001081169830285944
Epoch 4567, Loss: 0.017562348861240196, Final Batch Loss: 1.1920928244535389e-07
Epoch 4568, Loss: 0.003840008925180882, Final Batch Loss: 0.0
Epoch 4569, Loss: 0.0197920220089145, Final Batch Loss: 0.0
Epoch 4570, Loss: 0.03842209675349295, Final Batch Loss: 0.0
Epoch 4571, Loss: 0.022072176449000835, Final Batch Loss: 0.0
Epoch 4572, Loss: 0.013265793444560359, Final Batch Loss: 2.38

Epoch 4684, Loss: 0.004442470446520019, Final Batch Loss: 0.0
Epoch 4685, Loss: 0.05168070080981124, Final Batch Loss: 0.0001333863037871197
Epoch 4686, Loss: 0.02289834665134549, Final Batch Loss: 0.0
Epoch 4687, Loss: 0.003939905378501862, Final Batch Loss: 0.0
Epoch 4688, Loss: 0.012402359105180949, Final Batch Loss: 0.0
Epoch 4689, Loss: 0.002088884566546767, Final Batch Loss: 8.702239938429557e-06
Epoch 4690, Loss: 0.03127234138082713, Final Batch Loss: 0.0
Epoch 4691, Loss: 0.0037003081815782934, Final Batch Loss: 0.0
Epoch 4692, Loss: 0.0010637784871505573, Final Batch Loss: 0.0
Epoch 4693, Loss: 0.0020083064864593325, Final Batch Loss: 1.585470999998506e-05
Epoch 4694, Loss: 0.011584402425796725, Final Batch Loss: 0.0
Epoch 4695, Loss: 0.0021281463559148506, Final Batch Loss: 3.576278118089249e-07
Epoch 4696, Loss: 0.001652619219377982, Final Batch Loss: 1.1920928244535389e-07
Epoch 4697, Loss: 0.013095693226205185, Final Batch Loss: 0.01091910619288683
Epoch 4698, Loss: 0.0039

Epoch 4807, Loss: 0.0010435455696438112, Final Batch Loss: 3.576278118089249e-07
Epoch 4808, Loss: 0.03386653258348815, Final Batch Loss: 0.0
Epoch 4809, Loss: 0.011721412971269274, Final Batch Loss: 4.768370445162873e-07
Epoch 4810, Loss: 0.01697569907992147, Final Batch Loss: 0.015853434801101685
Epoch 4811, Loss: 0.004520756367128342, Final Batch Loss: 0.0
Epoch 4812, Loss: 0.005328663683023649, Final Batch Loss: 1.1920928244535389e-07
Epoch 4813, Loss: 0.0009560380131006241, Final Batch Loss: 0.0
Epoch 4814, Loss: 0.002774125852738507, Final Batch Loss: 0.0
Epoch 4815, Loss: 0.028571601695148274, Final Batch Loss: 0.0
Epoch 4816, Loss: 0.001128818679710264, Final Batch Loss: 1.1920928244535389e-07
Epoch 4817, Loss: 0.010721199389074343, Final Batch Loss: 1.1920928244535389e-07
Epoch 4818, Loss: 0.0008227453417930519, Final Batch Loss: 8.702239938429557e-06
Epoch 4819, Loss: 0.001888651677290909, Final Batch Loss: 0.0
Epoch 4820, Loss: 0.0010522570373723283, Final Batch Loss: 0.0
Ep

Epoch 4932, Loss: 0.012045142677322929, Final Batch Loss: 3.576278118089249e-07
Epoch 4933, Loss: 0.0011450903111835942, Final Batch Loss: 0.0
Epoch 4934, Loss: 0.00234323728363961, Final Batch Loss: 0.0
Epoch 4935, Loss: 0.003701043300679885, Final Batch Loss: 0.0
Epoch 4936, Loss: 0.003967753727920353, Final Batch Loss: 0.0
Epoch 4937, Loss: 0.006522424744616728, Final Batch Loss: 0.0
Epoch 4938, Loss: 0.01355735256220214, Final Batch Loss: 0.0
Epoch 4939, Loss: 0.00114951743353231, Final Batch Loss: 3.6954811548639555e-06
Epoch 4940, Loss: 0.007845774642191827, Final Batch Loss: 0.0
Epoch 4941, Loss: 0.002390411180385854, Final Batch Loss: 0.0
Epoch 4942, Loss: 0.012381578708300367, Final Batch Loss: 0.0
Epoch 4943, Loss: 0.011653181194560602, Final Batch Loss: 0.0
Epoch 4944, Loss: 0.004269608747563325, Final Batch Loss: 0.0
Epoch 4945, Loss: 0.003844088285404723, Final Batch Loss: 0.0
Epoch 4946, Loss: 0.013235429825726897, Final Batch Loss: 0.0
Epoch 4947, Loss: 0.001401471963617

In [10]:
softmax = nn.Softmax(dim = 1)
model.eval()
for batch in test_loader:
    features, labels = batch
    _, preds = torch.max(softmax(model(features.float())), dim = 1)
    print(metrics.confusion_matrix((labels).cpu(), preds.cpu()))
    print(metrics.classification_report((labels).cpu(), preds.cpu(), digits = 5))

[[79  0  0]
 [ 0 51  0]
 [ 0  0 58]]
              precision    recall  f1-score   support

           0    1.00000   1.00000   1.00000        79
           1    1.00000   1.00000   1.00000        51
           2    1.00000   1.00000   1.00000        58

    accuracy                        1.00000       188
   macro avg    1.00000   1.00000   1.00000       188
weighted avg    1.00000   1.00000   1.00000       188



In [12]:
gen = Generator(z_dim = 109)
load_model(gen, "3 Label 6 Subject GAN Ablation_gen.param")
gen.eval()

Generator(
  (gen): Sequential(
    (0): Sequential(
      (0): Linear(in_features=109, out_features=80, bias=True)
      (1): Dropout(p=0.1, inplace=False)
      (2): BatchNorm1d(80, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (3): ReLU(inplace=True)
    )
    (1): Sequential(
      (0): Linear(in_features=80, out_features=60, bias=True)
      (1): Dropout(p=0.1, inplace=False)
      (2): BatchNorm1d(60, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (3): ReLU(inplace=True)
    )
    (2): Sequential(
      (0): Linear(in_features=60, out_features=50, bias=True)
      (1): Dropout(p=0.1, inplace=False)
      (2): BatchNorm1d(50, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (3): ReLU(inplace=True)
    )
    (3): Linear(in_features=50, out_features=33, bias=True)
    (4): Tanh()
  )
)

In [33]:
size = len(X_test)
latent_vectors = get_noise(size, 100)
act_vectors = get_act_matrix(size, 3)
usr_vectors = get_usr_matrix(size, 6)

to_gen = torch.cat((latent_vectors, act_vectors[1], usr_vectors[1]), 1)
fake_features = gen(to_gen).detach()

In [18]:
_, preds = torch.max(softmax(model(fake_features.float())), dim = 1)
print(metrics.confusion_matrix(act_vectors[0], preds.cpu()))
print(metrics.classification_report(act_vectors[0], preds.cpu(), digits = 5))

[[56  0  0]
 [ 0 66  0]
 [ 0  0 66]]
              precision    recall  f1-score   support

           0    1.00000   1.00000   1.00000        56
           1    1.00000   1.00000   1.00000        66
           2    1.00000   1.00000   1.00000        66

    accuracy                        1.00000       188
   macro avg    1.00000   1.00000   1.00000       188
weighted avg    1.00000   1.00000   1.00000       188



# User Classifier

In [25]:
activities = [1, 3, 4]
users = [1, 3, 5, 7, 8, 11]

X, y = start_data(activities, users, "Subject", sub_features, act_features)

In [26]:
for k in range(len(y)):
    if y[k] == 1:
        y[k] = 0
    elif y[k] == 3:
        y[k] = 1
    elif y[k] == 5:
        y[k] = 2
    elif y[k] == 7:
        y[k] = 3
    elif y[k] == 8:
        y[k] = 4
    else:
        y[k] = 5

In [27]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, shuffle = True)

model_subject = Subject_Classifier()
lr = 0.001
n_epochs = 5000
batch_size = 250

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model_subject.parameters(), lr = lr)

train_features = torch.tensor(X_train)
train_labels = torch.tensor(y_train)
test_features = torch.tensor(X_test)
test_labels = torch.tensor(y_test)

train_data = torch.utils.data.TensorDataset(train_features, train_labels)
test_data = torch.utils.data.TensorDataset(test_features, test_labels)

train_loader = torch.utils.data.DataLoader(train_data, batch_size = batch_size, shuffle = True)
test_loader = torch.utils.data.DataLoader(test_data, batch_size = len(test_labels), shuffle = True)

In [28]:
for epoch in range(n_epochs):
    total_loss = 0
    for batch in train_loader:
        features, labels = batch
        
        optimizer.zero_grad()
        preds = model_subject(features.float())
        
        loss = criterion(preds, labels.long()) 
        loss.backward()
        
        optimizer.step()
        total_loss += loss.item()
        
    print(f'Epoch {epoch + 1}, Loss: {total_loss}, Final Batch Loss: {loss.item()}')

Epoch 1, Loss: 7.404765486717224, Final Batch Loss: 1.9325590133666992
Epoch 2, Loss: 7.557250142097473, Final Batch Loss: 2.090374708175659
Epoch 3, Loss: 7.312644720077515, Final Batch Loss: 1.848496675491333
Epoch 4, Loss: 7.331683874130249, Final Batch Loss: 1.872426986694336
Epoch 5, Loss: 7.413126349449158, Final Batch Loss: 1.9549418687820435
Epoch 6, Loss: 7.526399254798889, Final Batch Loss: 2.072183847427368
Epoch 7, Loss: 7.174553155899048, Final Batch Loss: 1.7197792530059814
Epoch 8, Loss: 7.491686820983887, Final Batch Loss: 2.0428950786590576
Epoch 9, Loss: 7.0883495807647705, Final Batch Loss: 1.6400468349456787
Epoch 10, Loss: 7.317397832870483, Final Batch Loss: 1.8740417957305908
Epoch 11, Loss: 7.121568202972412, Final Batch Loss: 1.6763677597045898
Epoch 12, Loss: 7.06843626499176, Final Batch Loss: 1.6258912086486816
Epoch 13, Loss: 7.374050736427307, Final Batch Loss: 1.9352648258209229
Epoch 14, Loss: 7.104016542434692, Final Batch Loss: 1.6648855209350586
Epoch

Epoch 121, Loss: 6.947214365005493, Final Batch Loss: 2.4900126457214355
Epoch 122, Loss: 6.141047716140747, Final Batch Loss: 1.697652816772461
Epoch 123, Loss: 5.0949559807777405, Final Batch Loss: 0.6629793047904968
Epoch 124, Loss: 5.471714615821838, Final Batch Loss: 1.0293619632720947
Epoch 125, Loss: 5.705200552940369, Final Batch Loss: 1.2358460426330566
Epoch 126, Loss: 5.6658406257629395, Final Batch Loss: 1.199222207069397
Epoch 127, Loss: 5.9078956842422485, Final Batch Loss: 1.5511351823806763
Epoch 128, Loss: 6.417731523513794, Final Batch Loss: 2.0361499786376953
Epoch 129, Loss: 5.400048673152924, Final Batch Loss: 0.9706173539161682
Epoch 130, Loss: 6.203579306602478, Final Batch Loss: 1.7060704231262207
Epoch 131, Loss: 5.72327446937561, Final Batch Loss: 1.1769216060638428
Epoch 132, Loss: 6.024537801742554, Final Batch Loss: 1.6313540935516357
Epoch 133, Loss: 5.973628044128418, Final Batch Loss: 1.6137535572052002
Epoch 134, Loss: 5.733196258544922, Final Batch Los

Epoch 240, Loss: 4.807971239089966, Final Batch Loss: 0.7923368215560913
Epoch 241, Loss: 6.722527384757996, Final Batch Loss: 2.722194194793701
Epoch 242, Loss: 5.089017629623413, Final Batch Loss: 1.0971534252166748
Epoch 243, Loss: 6.961426019668579, Final Batch Loss: 2.9538958072662354
Epoch 244, Loss: 5.4144017696380615, Final Batch Loss: 1.3670140504837036
Epoch 245, Loss: 5.162849187850952, Final Batch Loss: 1.0527048110961914
Epoch 246, Loss: 5.723467707633972, Final Batch Loss: 1.557281732559204
Epoch 247, Loss: 5.371070623397827, Final Batch Loss: 1.246679425239563
Epoch 248, Loss: 6.009191513061523, Final Batch Loss: 1.990105390548706
Epoch 249, Loss: 5.6194223165512085, Final Batch Loss: 1.4666857719421387
Epoch 250, Loss: 5.903215408325195, Final Batch Loss: 1.7328695058822632
Epoch 251, Loss: 5.053424775600433, Final Batch Loss: 0.8859243988990784
Epoch 252, Loss: 4.578644752502441, Final Batch Loss: 0.5144016742706299
Epoch 253, Loss: 5.858137726783752, Final Batch Loss:

Epoch 359, Loss: 4.970702052116394, Final Batch Loss: 1.0682365894317627
Epoch 360, Loss: 5.453144311904907, Final Batch Loss: 1.5506433248519897
Epoch 361, Loss: 4.83413553237915, Final Batch Loss: 1.026991605758667
Epoch 362, Loss: 6.461446762084961, Final Batch Loss: 2.641084671020508
Epoch 363, Loss: 4.958101868629456, Final Batch Loss: 1.1345170736312866
Epoch 364, Loss: 5.04952347278595, Final Batch Loss: 1.2324848175048828
Epoch 365, Loss: 5.182866454124451, Final Batch Loss: 1.3571943044662476
Epoch 366, Loss: 4.762534499168396, Final Batch Loss: 0.9867322444915771
Epoch 367, Loss: 5.216724753379822, Final Batch Loss: 1.441164493560791
Epoch 368, Loss: 5.35165536403656, Final Batch Loss: 1.4987585544586182
Epoch 369, Loss: 4.720380663871765, Final Batch Loss: 0.8880308866500854
Epoch 370, Loss: 4.853997349739075, Final Batch Loss: 0.9951685667037964
Epoch 371, Loss: 4.43034040927887, Final Batch Loss: 0.6410348415374756
Epoch 372, Loss: 5.039961218833923, Final Batch Loss: 1.28

Epoch 480, Loss: 4.64704704284668, Final Batch Loss: 1.1283979415893555
Epoch 481, Loss: 4.523630142211914, Final Batch Loss: 1.0623546838760376
Epoch 482, Loss: 3.996812105178833, Final Batch Loss: 0.5653464794158936
Epoch 483, Loss: 4.032197117805481, Final Batch Loss: 0.5587352514266968
Epoch 484, Loss: 5.650731325149536, Final Batch Loss: 2.1918740272521973
Epoch 485, Loss: 5.079882740974426, Final Batch Loss: 1.713343858718872
Epoch 486, Loss: 4.147972643375397, Final Batch Loss: 0.7941895127296448
Epoch 487, Loss: 6.485862135887146, Final Batch Loss: 3.084554672241211
Epoch 488, Loss: 4.574575185775757, Final Batch Loss: 1.0706889629364014
Epoch 489, Loss: 6.256928443908691, Final Batch Loss: 2.864274024963379
Epoch 490, Loss: 4.048253774642944, Final Batch Loss: 0.6201677322387695
Epoch 491, Loss: 4.654390215873718, Final Batch Loss: 1.216693639755249
Epoch 492, Loss: 4.500250339508057, Final Batch Loss: 1.0224665403366089
Epoch 493, Loss: 3.844268500804901, Final Batch Loss: 0.

...

Epoch 4435, Loss: 1.8690557181835175, Final Batch Loss: 0.25371119379997253
Epoch 4436, Loss: 3.6288733780384064, Final Batch Loss: 2.056915283203125
Epoch 4437, Loss: 1.6027365885674953, Final Batch Loss: 0.02169787511229515
Epoch 4438, Loss: 1.6056824922555961, Final Batch Loss: 1.0728830375228426e-06
Epoch 4439, Loss: 2.654489517211914, Final Batch Loss: 0.9713022112846375
Epoch 4440, Loss: 1.5718214093067218, Final Batch Loss: 0.0001554368354845792
Epoch 4441, Loss: 3.51875638961792, Final Batch Loss: 2.016861915588379
Epoch 4442, Loss: 1.6012707650661469, Final Batch Loss: 0.06629890203475952
Epoch 4443, Loss: 1.562618268537335, Final Batch Loss: 0.0016110072610899806
Epoch 4444, Loss: 1.5231858626939356, Final Batch Loss: 0.001717998180538416
Epoch 4445, Loss: 3.1167512238025665, Final Batch Loss: 1.6131900548934937
Epoch 4446, Loss: 2.2878766655921936, Final Batch Loss: 0.7909042239189148
Epoch 4447, Loss: 1.5289289305219427, Final Batch Loss: 0.0004345426568761468
Epoch 4448, L

Epoch 4554, Loss: 1.4945603661471978, Final Batch Loss: 0.0004102342063561082
Epoch 4555, Loss: 1.5282880887389183, Final Batch Loss: 0.030756168067455292
Epoch 4556, Loss: 1.5100734233856201, Final Batch Loss: 0.06579864025115967
Epoch 4557, Loss: 1.5273335622623563, Final Batch Loss: 0.015019779093563557
Epoch 4558, Loss: 1.761379063129425, Final Batch Loss: 0.33174848556518555
Epoch 4559, Loss: 2.3946887254714966, Final Batch Loss: 0.8848375678062439
Epoch 4560, Loss: 2.1114483177661896, Final Batch Loss: 0.6250478029251099
Epoch 4561, Loss: 1.618147850036621, Final Batch Loss: 0.17479407787322998
Epoch 4562, Loss: 1.3655972480773926, Final Batch Loss: 0.0
Epoch 4563, Loss: 1.4013914071256295, Final Batch Loss: 0.00158791767898947
Epoch 4564, Loss: 1.6079275906085968, Final Batch Loss: 0.2594066858291626
Epoch 4565, Loss: 1.8634423613548279, Final Batch Loss: 0.4667836129665375
Epoch 4566, Loss: 1.977048397064209, Final Batch Loss: 0.49567413330078125
Epoch 4567, Loss: 3.50013935565

Epoch 4670, Loss: 1.7023706957697868, Final Batch Loss: 0.03719734400510788
Epoch 4671, Loss: 3.308130979537964, Final Batch Loss: 1.707349419593811
Epoch 4672, Loss: 1.716741930693388, Final Batch Loss: 0.026943515986204147
Epoch 4673, Loss: 1.6325099989771843, Final Batch Loss: 0.009257368743419647
Epoch 4674, Loss: 1.9009304344654083, Final Batch Loss: 0.26632383465766907
Epoch 4675, Loss: 1.6519737225025892, Final Batch Loss: 0.005459992215037346
Epoch 4676, Loss: 1.5626790383830667, Final Batch Loss: 0.011061654426157475
Epoch 4677, Loss: 5.068981885910034, Final Batch Loss: 3.5851430892944336
Epoch 4678, Loss: 1.8717913925647736, Final Batch Loss: 0.3354591131210327
Epoch 4679, Loss: 1.6220767344348133, Final Batch Loss: 0.006959601771086454
Epoch 4680, Loss: 2.4459195733070374, Final Batch Loss: 0.9089332222938538
Epoch 4681, Loss: 1.5191039741032455, Final Batch Loss: 1.1920922133867862e-06
Epoch 4682, Loss: 1.515822820365429, Final Batch Loss: 0.011957041919231415
Epoch 4683, 

Epoch 4785, Loss: 1.5706231407821178, Final Batch Loss: 0.023399990051984787
Epoch 4786, Loss: 2.8826266527175903, Final Batch Loss: 1.4083406925201416
Epoch 4787, Loss: 1.4979877099394798, Final Batch Loss: 0.06618934124708176
Epoch 4788, Loss: 1.5336046479642391, Final Batch Loss: 0.016662713140249252
Epoch 4789, Loss: 2.198865056037903, Final Batch Loss: 0.7098842263221741
Epoch 4790, Loss: 1.407034957781434, Final Batch Loss: 0.007017726078629494
Epoch 4791, Loss: 1.842935860157013, Final Batch Loss: 0.3108447194099426
Epoch 4792, Loss: 3.1018348336219788, Final Batch Loss: 1.6335625648498535
Epoch 4793, Loss: 2.571726471185684, Final Batch Loss: 1.0466399192810059
Epoch 4794, Loss: 1.5894757211208344, Final Batch Loss: 0.03108811378479004
Epoch 4795, Loss: 1.8128736019134521, Final Batch Loss: 0.14642935991287231
Epoch 4796, Loss: 2.4750254154205322, Final Batch Loss: 0.835788369178772
Epoch 4797, Loss: 2.786014139652252, Final Batch Loss: 1.2395128011703491
Epoch 4798, Loss: 1.46

Epoch 4900, Loss: 1.5246775671839714, Final Batch Loss: 0.07367666810750961
Epoch 4901, Loss: 1.4847206268459558, Final Batch Loss: 0.011229762807488441
Epoch 4902, Loss: 1.5214804112911224, Final Batch Loss: 0.1637032926082611
Epoch 4903, Loss: 2.907282531261444, Final Batch Loss: 1.4947311878204346
Epoch 4904, Loss: 2.8861744105815887, Final Batch Loss: 1.3833611011505127
Epoch 4905, Loss: 3.5488114058971405, Final Batch Loss: 2.0766184329986572
Epoch 4906, Loss: 1.4905315963551402, Final Batch Loss: 0.004504295997321606
Epoch 4907, Loss: 1.5096422149799764, Final Batch Loss: 0.0033004595898091793
Epoch 4908, Loss: 1.456504476169357, Final Batch Loss: 0.0003084660565946251
Epoch 4909, Loss: 1.4409967957526533, Final Batch Loss: 1.6689160474925302e-05
Epoch 4910, Loss: 2.8047457337379456, Final Batch Loss: 1.4399470090866089
Epoch 4911, Loss: 1.3971064604411367, Final Batch Loss: 0.00027581225731410086
Epoch 4912, Loss: 1.4564647375946151, Final Batch Loss: 1.0847986231965479e-05
Epoc

In [29]:
softmax = nn.Softmax(dim = 1)
model_subject.eval()
for batch in test_loader:
    features, labels = batch
    _, preds = torch.max(softmax(model_subject(features.float())), dim = 1)
    print(metrics.confusion_matrix((labels).cpu(), preds.cpu()))
    print(metrics.classification_report((labels).cpu(), preds.cpu(), digits = 5))

[[27  0  0  0  1  3]
 [ 0 33  0  6  0  0]
 [ 0  0 21  5  0  1]
 [ 0  0  0 32  0  0]
 [ 5  5  0  0 10  2]
 [ 0  0  0  1  0 36]]
              precision    recall  f1-score   support

           0    0.84375   0.87097   0.85714        31
           1    0.86842   0.84615   0.85714        39
           2    1.00000   0.77778   0.87500        27
           3    0.72727   1.00000   0.84211        32
           4    0.90909   0.45455   0.60606        22
           5    0.85714   0.97297   0.91139        37

    accuracy                        0.84574       188
   macro avg    0.86761   0.82040   0.82481       188
weighted avg    0.86176   0.84574   0.83844       188



In [34]:
_, preds = torch.max(softmax(model_subject(fake_features.float())), dim = 1)
print(metrics.confusion_matrix(usr_vectors[0], preds.cpu()))
print(metrics.classification_report(usr_vectors[0], preds.cpu(), digits = 5))

[[27  0  0  0  0  0]
 [ 0 32  0  0  0  0]
 [ 0  0 30  0  0  8]
 [ 0  0  0 30  0  0]
 [17  0  0  0  0  7]
 [ 0  0  0  0  0 37]]
              precision    recall  f1-score   support

           0    0.61364   1.00000   0.76056        27
           1    1.00000   1.00000   1.00000        32
           2    1.00000   0.78947   0.88235        38
           3    1.00000   1.00000   1.00000        30
           4    0.00000   0.00000   0.00000        24
           5    0.71154   1.00000   0.83146        37

    accuracy                        0.82979       188
   macro avg    0.72086   0.79825   0.74573       188
weighted avg    0.76008   0.82979   0.78100       188



C:\Users\hkimr\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\hkimr\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\hkimr\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
